<a href="https://colab.research.google.com/github/ecordeiro/cefet/blob/main/pymoo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#!pip install pymoo

Class development

In [ ]:
import numpy as np
from pymoo.core.problem import ElementwiseProblem
from pymoo.core.problem import Problem
import math

In [ ]:
class ZDT1(Problem):
    def __init__(self):
        super().__init__(n_var=30, n_obj=2, n_constr=0, xl=0.0, xu=1.0, type_var=np.double)

    def _evaluate(self, x, out, *args, **kwargs):
        f1 = x[:, 0]
        g = 1 + 9 * np.mean(x[:, 1:], axis=1)
        f2 = g * (1 - np.sqrt(f1 / g))

        out["F"] = np.column_stack([f1, f2])

# Para criar uma instância do problema ZDT1
problem = ZDT1()

In [ ]:
class MyProblem(ElementwiseProblem):

    def __init__(self):
        super().__init__(n_Var = 2,
                         n_obj = 2,
                         n_ieq_constr = 2,
                         xl = np.array([0,0]),
                         xu = np.array([1,1])
                         )

    def r_g(self, x):
        """
        Calcula a função g(x_2, ..., x_D)

        Args:
            x: Uma lista ou array contendo os valores x_2, ..., x_D.

        Returns:
            O valor calculado da função g.
        """
        D = len(x)+1  # Calcula o valor de D com base no tamanho da lista
        soma = sum(x_d / (D - 1) for x_d in x)  # Calcula o somatório
        return 1 + 9 * soma


    def r_h(self, f1, g):
        """Calcula o valor da função h(f1, g) = 1 - sqrt(f1/g).

        Args:
            f1: O valor do primeiro argumento.
            g: O valor do segundo argumento (deve ser diferente de zero).

        Returns:
            O valor da função h(f1, g).
        """

        if g == 0:
            raise ValueError("O segundo argumento (g) não pode ser zero.")

        return 1 - math.sqrt(f1 / g)

    def _evaluate(self, x, out, *args, **kwargs):

        f1 = x[0]
        g = r_g(x)
        h = r_h(f1,g)
        f2 = g*h

        out["F"] = np.column_stack([f1, f2])


problem = MyProblem()

In [ ]:
#from pymoo.problems import get_problem

In [ ]:
#problem = get_problem("zdt1")

Initializing the Algorithm

In [ ]:
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.operators.sampling.rnd import FloatRandomSampling
from pymoo.operators.crossover.sbx import SBX
from pymoo.operators.mutation.pm import PM

In [ ]:
algorithm = NSGA2(
    pop_size=200,
    # n_offsprings = 50,
    # sampling = FloatRandomSampling(),
    # crossover = SBX(prob=0.8, eta=20),
    # mutation = PM(eta = 25),
    # eliminate_duplicates = True
)

In [ ]:
from pymoo.termination import get_termination
termination = get_termination("n_gen",100)

Optimization Process

In [ ]:
from pymoo.optimize import minimize

res = minimize(
    problem,
    algorithm,
    termination,
    seed =  7,
    seve_history = True,
    verbose = True
)

ValueError: negative dimensions are not allowed